In [3]:
%pylab inline
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV , RandomizedSearchCV
from sklearn.metrics import classification_report
from scipy import stats
from sklearn.ensemble import RandomForestClassifier

Populating the interactive namespace from numpy and matplotlib


In [4]:
cd Test05mw80km30OSNR/

C:\Users\ChiefGuti\Documents\Lund University\Thesis\MLImplementacion\Nonlinear fiber\Dispersion Manged fiber\Test05mw80km30OSNR


In [12]:
def visualize_classifier(model, X, y,namef, ax=None, cmap='tab20b'):
    ax = ax or plt.gca()
 


    # Plot the training points
    ax.scatter(X[:, 0], X[:, 1], c=y, marker='.', cmap=cmap,
               clim=(y.min(), y.max()), zorder=3)
    ax.axis('tight')
    ax.axis('off')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # fit the estimator
    model.fit(X, y)
    xx, yy = np.meshgrid(np.linspace(*xlim, num=500),
                         np.linspace(*ylim, num=500))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    # Create a color plot with the results
    n_classes = len(np.unique(y))
    contours = ax.contourf(xx, yy, Z, alpha=0.3,
                           levels=np.arange(n_classes + 1) - 0.5,
                           cmap=cmap, clim=(y.min(), y.max()),
                           zorder=1)

    ax.set(xlim=xlim, ylim=ylim)
    savefig(namef,dpi=300)
    close()

In [13]:
def NormData(DataSet,data):
    for i in range(len(data[:,0])):
        DataSet[i,0]=data[:,0][i]
        DataSet[i,1]=data[:,1][i]
    DataSet[:,0]=(DataSet[:,0]-min(DataSet[:,0]))/(max(DataSet[:,0])-min(DataSet[:,0]))
    DataSet[:,1]=(DataSet[:,1]-min(DataSet[:,1]))/(max(DataSet[:,1])-min(DataSet[:,1]))
def ColorMap(Word,Map,M):
    Cmap=arange(M)
    for i in range(len(Word)):
        for j in range(len(Cmap)):
            if(Word[i]==Org[j]):
                Map[i]=Cmap[j]

def BER(x,y):
    j=0
    for i in range(len(x)):
        if(x[i]!=y[i]):
            j+=1
    return(j/len(x))
            

In [14]:
Code=loadtxt('PBRS.txt',skiprows=5)

In [15]:
INT=[]
for i in range(int(len(Code)/4)):
    INT.append(i*4)
CodeWord=[]
for i in range(len(Code)):
    CodeWord.append(str(int(Code[i])))

Word=[]
Org=[]
j=0
for i in INT:
    Word.append(CodeWord[i]+CodeWord[i+1]+CodeWord[i+2]+CodeWord[i+3])
    
    if((Word[j] in Org)==False ):
        Org.append(Word[j])
    j=j+1
Map=zeros(len(Word))
ColorMap(Word,Map,16)

In [16]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 500, num = 1)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 20, num = 1)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1,2]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(random_grid)


{'max_depth': [5, None], 'max_features': ['auto', 'sqrt'], 'n_estimators': [200], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2]}


In [21]:
N=arange(0,10)
SERforest=zeros(10)
SERforestmit=zeros(10)
best=[]
bestmit=[]
for i in range(len(N)):
    data = loadtxt('N%iL80X.txt'%(N[i]))
    datamit=[]
    datamit.append(data[:,2])
    datamit.append(data[:,3])
    datamit=transpose(datamit)
    DataSet=zeros((len(data[:,0]),2))
    NormData(DataSet,data)
    DataSetmit=zeros((len(data[:,0]),2))
    NormData(DataSetmit,datamit)
    scatter(DataSet[:,0],DataSet[:,1],c=Map,s=1,cmap='tab20b')
    tick_params(axis='both', which='both',length=0)
    savefig('con%i.png'%i,dpi=250)
    close()
    X_train, X_test, y_train, y_test = train_test_split(DataSet, Map, test_size = 0.20)
    gridF = GridSearchCV(RandomForestClassifier(), random_grid, cv = 2, verbose = 1, n_jobs = -1)
    bestF = gridF.fit(X_train, y_train)
    best.append(gridF.best_params_)
    forest=RandomForestClassifier(max_depth=best[i]['max_depth'],max_features=best[i]['max_features'],min_samples_leaf=best[i]['min_samples_leaf'],min_samples_split=best[i]['min_samples_split'],n_estimators=best[i]['n_estimators'])
    forest.fit(X_train,y_train)
    y_prediction=forest.predict(X_test)
    visualize_classifier(forest,X_train,y_train,'conDes%i.png'%i)
    SERforest[i]=BER(y_test,y_prediction)
    Xmit_train, Xmit_test, ymit_train, ymit_test = train_test_split(DataSetmit, Map, test_size = 0.20)
    gridFmit = GridSearchCV(RandomForestClassifier(), random_grid, cv = 2, verbose = 1, n_jobs = -1)
    bestFmit = gridFmit.fit(Xmit_train, ymit_train)
    bestmit.append(gridFmit.best_params_)
    forestmit=RandomForestClassifier(max_depth=bestmit[i]['max_depth'],max_features=bestmit[i]['max_features'],min_samples_leaf=bestmit[i]['min_samples_leaf'],min_samples_split=bestmit[i]['min_samples_split'],n_estimators=bestmit[i]['n_estimators'])
    forestmit.fit(Xmit_train,ymit_train)
    ymit_prediction=forestmit.predict(Xmit_test)
    SERforestmit[i]=BER(ymit_test,ymit_prediction)


Fitting 2 folds for each of 24 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:   16.2s finished


Fitting 2 folds for each of 24 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:   16.8s finished


Fitting 2 folds for each of 24 candidates, totalling 48 fits


...

[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:   16.6s finished


Fitting 2 folds for each of 24 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:   19.2s finished


Fitting 2 folds for each of 24 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:   17.6s finished


In [22]:
SERforest

array([ 0.05365854,  0.42926829,  0.66341463,  0.79268293,  0.8195122 ,
        0.8195122 ,  0.83902439,  0.8804878 ,  0.82195122,  0.84634146])

In [23]:
SERforestmit

array([ 0.07317073,  0.48780488,  0.71707317,  0.83658537,  0.89268293,
        0.88292683,  0.92439024,  0.88536585,  0.88536585,  0.91707317])